In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/11/22 00:51:02 WARN Utils: Your hostname, codespaces-a019c6 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/11/22 00:51:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/22 00:51:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
rdd.take(10)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 9, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 11, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 16, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 12, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 15, 22, 42), PULocationID=166, total_amount=12.74),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 7, 7, 42), PULocationID=179, total_amount=5.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 4, 22, 29), PULocationID=41, total_amount=25.05),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 13, 5, 28), PULocationID=75, total_amount=21.33),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 10, 51, 37), PULocationID=152, total_amount=7.8),
 Row(lpep_pickup_datetime=datetime.datetime

In [5]:
rdd.filter(lambda row: False).take(1)

[]

In [6]:
from datetime import datetime

In [7]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [8]:
rdd.filter(filter_outliers).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 9, 10, 15), PULocationID=74, total_amount=44.97)]

In [9]:
rows = rdd.take(10)
row = rows[0]

In [10]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 9, 10, 15), PULocationID=74, total_amount=44.97)

In [11]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 23, 9, 0)

In [12]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [13]:
rdd.filter(filter_outliers).\
    map(prepare_for_grouping).\
        take(10)

[((datetime.datetime(2020, 1, 23, 9, 0), 74), (44.97, 1)),
 ((datetime.datetime(2020, 1, 20, 11, 0), 67), (33.45, 1)),
 ((datetime.datetime(2020, 1, 15, 16, 0), 260), (8.3, 1)),
 ((datetime.datetime(2020, 1, 5, 12, 0), 82), (8.3, 1)),
 ((datetime.datetime(2020, 1, 29, 15, 0), 166), (12.74, 1)),
 ((datetime.datetime(2020, 1, 15, 7, 0), 179), (5.8, 1)),
 ((datetime.datetime(2020, 1, 16, 4, 0), 41), (25.05, 1)),
 ((datetime.datetime(2020, 1, 28, 13, 0), 75), (21.33, 1)),
 ((datetime.datetime(2020, 1, 22, 10, 0), 152), (7.8, 1)),
 ((datetime.datetime(2020, 1, 31, 6, 0), 75), (26.05, 1))]

In [14]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [15]:
rdd.filter(filter_outliers).\
    map(prepare_for_grouping).\
    reduceByKey(calculate_revenue).\
        take(10)

[((datetime.datetime(2020, 1, 15, 7, 0), 179), (50.5, 5)),
 ((datetime.datetime(2020, 1, 2, 4, 0), 66), (197.69, 10)),
 ((datetime.datetime(2020, 1, 3, 17, 0), 196), (163.06, 8)),
 ((datetime.datetime(2020, 1, 16, 9, 0), 14), (163.73999999999998, 6)),
 ((datetime.datetime(2020, 1, 20, 14, 0), 116), (104.80999999999999, 9)),
 ((datetime.datetime(2020, 1, 30, 10, 0), 116), (94.33, 8)),
 ((datetime.datetime(2020, 1, 18, 3, 0), 55), (48.3, 1)),
 ((datetime.datetime(2020, 1, 31, 17, 0), 41), (588.1600000000001, 40)),
 ((datetime.datetime(2020, 1, 28, 6, 0), 76), (237.26, 10)),
 ((datetime.datetime(2020, 1, 23, 5, 0), 74), (1315.669999999999, 103))]

In [18]:
from collections import namedtuple

In [19]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [20]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [21]:
rdd.filter(filter_outliers).\
    map(prepare_for_grouping).\
    reduceByKey(calculate_revenue).\
    map(unwrap).\
        take(10)

[RevenueRow(hour=datetime.datetime(2020, 1, 15, 7, 0), zone=179, revenue=50.5, count=5),
 RevenueRow(hour=datetime.datetime(2020, 1, 2, 4, 0), zone=66, revenue=197.69, count=10),
 RevenueRow(hour=datetime.datetime(2020, 1, 3, 17, 0), zone=196, revenue=163.06, count=8),
 RevenueRow(hour=datetime.datetime(2020, 1, 16, 9, 0), zone=14, revenue=163.73999999999998, count=6),
 RevenueRow(hour=datetime.datetime(2020, 1, 20, 14, 0), zone=116, revenue=104.80999999999999, count=9),
 RevenueRow(hour=datetime.datetime(2020, 1, 30, 10, 0), zone=116, revenue=94.33, count=8),
 RevenueRow(hour=datetime.datetime(2020, 1, 18, 3, 0), zone=55, revenue=48.3, count=1),
 RevenueRow(hour=datetime.datetime(2020, 1, 31, 17, 0), zone=41, revenue=588.1600000000001, count=40),
 RevenueRow(hour=datetime.datetime(2020, 1, 28, 6, 0), zone=76, revenue=237.26, count=10),
 RevenueRow(hour=datetime.datetime(2020, 1, 23, 5, 0), zone=74, revenue=1315.669999999999, count=103)]

In [22]:
from pyspark.sql import types

In [23]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [24]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [25]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-15 11:00:00| 179|              50.5|    5|
|2020-01-02 08:00:00|  66|            197.69|   10|
|2020-01-03 21:00:00| 196|            163.06|    8|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-20 18:00:00| 116|104.80999999999999|    9|
|2020-01-30 14:00:00| 116|             94.33|    8|
|2020-01-18 07:00:00|  55|              48.3|    1|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-28 10:00:00|  76|            237.26|   10|
|2020-01-23 09:00:00|  74| 1315.669999999999|  103|
|2020-01-23 12:00:00| 119|             51.89|    2|
|2020-01-13 09:00:00|   7| 403.7600000000001|   29|
|2020-01-10 18:00:00| 260|325.51000000000005|   22|
|2020-01-24 06:00:00| 127| 79.50999999999999|    2|
|2020-01-06 16:00:00| 242|            246.31|   10|
|2020-01-12 00:00:00| 127|              17.8|    1|
|2020-01-08 

In [26]:
df_result.write.parquet('tmp/green-revenue')

In [27]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-23 13:10:15|  2020-01-23 13:38:16|                 N|         1|          74|         130|              1|        12.77|       36.0|  0.0|    0.

In [28]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [30]:
import pandas as pd

In [29]:
rows = duration_rdd.take(10)

In [31]:
rows

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 9, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 11, 9), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 16, 23, 41), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 12, 32, 26), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 15, 22, 42), PULocationID=166, DOLocationID=42, trip_distance=1.84),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 7, 7, 42), PULocationID=179, DOLocationID=223, trip_distance=0.76),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 4, 22, 29), PULocationID=41, DOLocationID=237, trip_distance=3.32),
 Row(VendorID=2, lpep_pickup_datetime=dat

In [33]:
def apply_batch_test(partition):
    cnt = 0
    for row in partition:
        cnt += 1
    return [cnt]

In [34]:
rdd.mapPartitions(apply_batch_test).collect()

[1164928, 456935, 425873, 256781]

In [35]:
df = pd.DataFrame(rows, columns=columns)

In [36]:
df

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2.0,2020-01-23 09:10:15,74,130,12.77
1,NaN,2020-01-20 11:09:00,67,39,8.00
2,2.0,2020-01-15 16:23:41,260,157,1.27
3,2.0,2020-01-05 12:32:26,82,83,1.25
4,2.0,2020-01-29 15:22:42,166,42,1.84
5,2.0,2020-01-15 07:07:42,179,223,0.76
6,2.0,2020-01-16 04:22:29,41,237,3.32
7,2.0,2020-01-28 13:05:28,75,161,2.21
8,1.0,2020-01-22 10:51:37,152,166,0.90
9,2.0,2020-01-31 06:25:04,75,234,6.10


In [37]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [38]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [40]:
# yield
def infinite_seq():
    i = 0

    while True:
        yield i
        i += 1

In [42]:
seq = infinite_seq()
print(next(seq))
print(next(seq))
print(next(seq))

0
1
2


In [39]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [43]:
duration_rdd.mapPartitions(apply_model_in_batch).take(10)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 09:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77, predicted_duration=63.849999999999994),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 11:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0, predicted_duration=40.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 16:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27, predicted_duration=6.35),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 12:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25, predicted_duration=6.25),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 15:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84, predicted_duration=9.200000000000001),
 Pandas(Index=5, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 07:07:42'), PULocationID=179, DOLocationID=223, trip_distance=0.7

In [44]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [45]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|63.849999999999994|
|              40.0|
|              6.35|
|              6.25|
| 9.200000000000001|
|               3.8|
|16.599999999999998|
|             11.05|
|               4.5|
|              30.5|
|               8.7|
|5.8999999999999995|
|              11.0|
|              15.2|
|              4.25|
|25.299999999999997|
|7.8500000000000005|
|              34.0|
| 5.300000000000001|
|              6.15|
+------------------+
only showing top 20 rows

